# Master Workflow Manager (Single Potential)

This is meant as a master workflow manager for setting up, running, and analyzing all implemented calculations for a single potential.  It is meant for helping evaluate the properties of a single potential and to serve as a guide for the steps in running the calculations across multiple potentials. 

## 1. Import Python packages

Calculation imports 

In [72]:
# Standard library imports
from __future__ import print_function, division
import os
import sys
import glob
import uuid
import shutil
from math import floor

# http://www.numpy.org/
import numpy as np

# http://pandas.pydata.org/
import pandas as pd

# http://ipython.org/
from IPython.display import display, HTML 

# https://github.com/usnistgov/DataModelDict
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

Plotting imports

In [2]:
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.embed import components
from bokeh.resources import Resources
from bokeh.io import output_notebook
print('bokeh version', bokeh.__version__)
output_notebook()

bokeh version 0.12.4


Loading BokehJS ...

## 2. Input parameters

### Specify run parameters

- **potential** (*str*) - The potential to investigate.
- **family** (*str*) - The family name (i.e. crystal prototype) corresponding to the primary crystal structure to examine.
- **database** (*str*) - The name of the database to interact with (defined [here](#databases)).
- **run_terms** (*dict*) - Defines the commands and directories to use for each number of processors on which this computer can run.
    
    - **lammps_command** (*str*) -The LAMMPS executable to use.
    - **mpi_command** (*str*) - The MPI command to call when LAMMPS is executed.
    - **run_directory** (*str*) - The run_directory to prepare the calculations in (defined with databases below).

In [3]:
potential = '2009--Purja-Pun-G-P--Ni-Al'
family = 'A1--Cu--fcc'
database = 'test'

run_terms = {}

# Define serial run terms
run_terms[1] = {}
run_terms[1]['lammps_command'] = 'lmp_mpi'
run_terms[1]['mpi_command'] = ''

# Define 2 core run terms
run_terms[2] = {}
run_terms[2]['lammps_command'] = 'lmp_mpi'
run_terms[2]['mpi_command'] = 'mpiexec -localonly 2'

# Define 3 core run terms
run_terms[3] = {}
run_terms[3]['lammps_command'] = 'lmp_mpi'
run_terms[3]['mpi_command'] = 'mpiexec -localonly 3'

# Define 4 core run terms
run_terms[4] = {}
run_terms[4]['lammps_command'] = 'lmp_mpi'
run_terms[4]['mpi_command'] = 'mpiexec -localonly 4'

### Define databases and run directories <a id="databases"></a>

These are specific to the computer this Notebook is running on and the databases that are being accessed.

In [4]:
# 'local' is a local directory
if database == 'local':
    dbase = iprPy.Database('local', 
                           host='C:\\Users\\lmh1\\Documents\\calculations\\ipr\\library')
    
    run_terms[1]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\local\\1'
    run_terms[2]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\local\\2'
    run_terms[3]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\local\\3'
    run_terms[4]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\local\\4'

# 'test' is a local directory for testing
elif database == 'test':
    dbase = iprPy.Database('local', 
                           host='C:\\Users\\lmh1\\Documents\\calculations\\ipr\\library_test')
    
    run_terms[1]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\test\\1'
    run_terms[2]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\test\\2'
    run_terms[3]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\test\\3'
    run_terms[4]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\test\\4'

# 'curator' is a local MDCS curator
elif database == 'curator':
    dbase = iprPy.Database('curator', 
                           host='https://iprhub.nist.gov/', 
                           user='admin', pswd='admin')
    
    run_terms[1]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\curator\\1'
    run_terms[2]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\curator\\2'
    run_terms[3]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\curator\\3'
    run_terms[4]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\curator\\4'

# 'iprhub' is the remote MDCS curator at iprhub
elif database == 'iprhub':
    dbase = iprPy.Database('curator', 
                           host='https://iprhub.nist.gov/', 
                           user='lmh1', 
                           pswd='C:/users/lmh1/documents/iprhub/iprhub_password.txt',
                           cert='C:/users/lmh1/documents/iprhub/iprhub-ca.pem')
    
    run_terms[1]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\iprhub\\1'
    run_terms[2]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\iprhub\\2'
    run_terms[3]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\iprhub\\3'
    run_terms[4]['run_directory'] = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\iprhub\\4'

## 3. Setup

### Verify iprPy modules loaded

In [5]:
iprPy.check_modules()

calculations that passed import:
 point_defect_static
 dynamic_relax
 surface_energy
 E_vs_r_scan
 dislocation_monopole
 stacking_fault
 refine_structure
 stacking_fault_multi
 LAMMPS_ELASTIC
calculations that failed import:

records that passed import:
 calculation_cohesive_energy_relation
 stacking_fault
 point_defect
 calculation_system_relax
 calculation_dislocation_monopole
 calculation_point_defect_formation
 dislocation_monopole
 calculation_generalized_stacking_fault
 free_surface
 crystal_prototype
 calculation_dynamic_relax
 calculation_surface_energy
 calculation_stacking_fault
 potential_LAMMPS
records that failed import:

databases that passed import:
 local
 curator
databases that failed import:



### Build database reference records

In [6]:
iprPy.highthroughput.build(dbase)

In [7]:
crystal_prototype_df = dbase.get_records_df(style='crystal_prototype')

counts = {}
for i, prototype in crystal_prototype_df.iterrows():
    model = DM(dbase.get_record(name=prototype.id).content)
    counts[prototype.id] = np.unique(model.finds('component'), return_counts=True)[1]

### Define functions and resources

**comp_refine** generates a composition string based on symbols and unique sites of prototypes

In [8]:
def comp_refine(symbols, counts):
    """Takes a list of symbols and count of how many times each symbol appears and generates a composition string."""
    primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47]
    
    sym_dict = {}
    for i in xrange(len(symbols)):
        sym_dict[symbols[i]] = counts[i]
    
    for prime in primes:
        if max(sym_dict.values()) < prime:
            break
        
        while True:
            breaktime = False
            for value in sym_dict.values():
                if value % prime != 0:
                    breaktime = True
                    break
            if breaktime:
                break
            for key in sym_dict:
                sym_dict[key] /= prime
    
    composition=''
    for key in sorted(sym_dict):
        if sym_dict[key] > 0:
            composition += key
            if sym_dict[key] != 1:
                composition += str(int(sym_dict[key]))
            
    return composition   

**plot_gen** generates the bokeh E vs. r plots.

In [21]:
def plot_gen(df, composition, colors, dashes):
    
    ymin = floor(df.min().min())
    if ymin < -10: ymin = -10
    
    p = figure(title='Cohesive Energy vs. Interatomic Spacing for ' + composition,
               plot_width = 800,
               plot_height = 600,
               x_range = [2, 6],
               y_range = [ymin, 0],              
               x_axis_label='r (A)', 
               y_axis_label='Cohesive Energy (eV/atom)')
   
    for prototype, values in df.iteritems():
        if prototype != 'r':
            p.line(df.r, values, 
                   legend=prototype, 
                   line_color=colors[prototype], 
                   line_dash=dashes[prototype], 
                   line_width = 2)            
    
    p.legend.location = "bottom_right"    
    return p  

max_string_length = 16

line_color = {
   #elemental
    'A1--Cu--fcc':                'black',
    'A2--W--bcc':                 'blue',
    'A3--Mg--hcp':                'red',
    'A3\'--alpha-La':             'cyan',
    'A4--C--dc':                  'magenta',
    'A5--beta-Sn':                '#EAC117',
    'A6--In--bct':                'orange',
    'A7--alpha-As':               'gray',
    'A15--beta-W':                'green',
    'Ah--alpha-Po--sc':           'brown',
   #1:1
    'B1--NaCl':                   'black',
    'B2--CsCl':                   'blue',
    'B3--ZnS':                    'red',
    'L1_0--AuCu':                 'cyan',
   #1:2
    'C1--CaF2':                   'black',
   #1:3
    'A15--Cr3Si':                 'black',
    'D0_3--BiF3':                 'blue',
    'L1_2--AuCu3':                'red',
   #1:1:2
    'L2_1--AlCu2Mn':              'red'
}

line_dash = {
   #elemental
    'A1--Cu--fcc':                'solid',
    'A2--W--bcc':                 'solid',
    'A3--Mg--hcp':                'dashed',
    'A3\'--alpha-La':             'dashdot',
    'A4--C--dc':                  'solid',
    'A5--beta-Sn':                'solid',
    'A6--In--bct':                'solid',
    'A7--alpha-As':               'solid',
    'A15--beta-W':                'solid',
    'Ah--alpha-Po--sc':           'solid',
   #1:1
    'B1--NaCl':                   'solid',
    'B2--CsCl':                   'solid',
    'B3--ZnS':                    'solid',
    'L1_0--AuCu':                 'solid',
   #1:2
    'C1--CaF2':                   'solid',
   #1:3
    'A15--Cr3Si':                 'solid',
    'D0_3--BiF3':                 'solid',
    'L1_2--AuCu3':                'solid',
   #1:1:2
    'L2_1--AlCu2Mn':              'solid'
}

## 4. E_vs_r_scan

In [10]:
n = 1
calculation_style = 'E_vs_r_scan'
record_style = 'calculation_cohesive_energy_relation'

### Prepare

In [11]:
run_directory = run_terms[n]['run_directory']

input_dict = {}
input_dict['calculation_style'] = calculation_style
input_dict['lammps_command'] = run_terms[n]['lammps_command']
input_dict['mpi_command'] = run_terms[n]['mpi_command']
input_dict['potential_name'] = potential

iprPy.highthroughput.prepare(dbase, run_directory, input_dict=input_dict)

### Run

In [12]:
iprPy.highthroughput.runner(dbase, run_directory)

### Analyze

Fetch records

In [13]:
results_df = dbase.get_records_df(style=record_style, full=True, flat=False)
results_df = results_df[results_df.potential_id == potential]
results_df = results_df[results_df.calc_script == 'calc_'+calculation_style]

print(len(results_df), 'records found')

60 records found


Check errors

In [14]:
error_df = results_df[results_df.status=='error']
print(len(error_df), 'issued errors')

if len(error_df) > 0:
    for error in np.unique(error_df.error.values):
        print(error)    

0 issued errors


Define composition

In [15]:
compositions = []
for i, calc in results_df.iterrows():
    compositions.append(comp_refine(calc.symbols, counts[calc.family]))
results_df = results_df.assign(composition=compositions)

Generate plots

In [23]:
# Loop over all compositions
for composition in np.unique(results_df.composition):
    composition_df = results_df[results_df.composition==composition]
    
    #Ignore false compounds (ones where number of unique symbols != number of symbols)
    composition_df = composition_df[(composition_df.symbols.apply(lambda x: len(np.unique(x))) 
                                     == composition_df.symbols.apply(lambda x: len(x)))] 
    
    #Extract plot values
    plot_df = {}
    columns = []
    plot_df['r'] = uc.get_in_units(composition_df.iloc[0].e_vs_r_plot.r, 'angstrom')
    for i, v in composition_df.iterrows():
        if len(v.family) > max_string_length:
            name = '--'.join(v.family.split('--')[:-1])
        else:
            name = v.family
        plot_df[name] = uc.get_in_units(v.e_vs_r_plot.E_coh, 'eV')
        columns.append(name)
    
    plot_df = pd.DataFrame(plot_df, columns=['r'] + sorted(columns))
    
    #Construct plot
    plot = plot_gen(plot_df, composition, line_color, line_dash)
    show(plot)

## 4. refine_structure and/or LAMMPS_ELASTIC

In [77]:
n = 1
#calculation_style = 'refine_structure'
calculation_style = 'LAMMPS_ELASTIC'
record_style = 'calculation_system_relax'

### Prepare

In [34]:
run_directory = run_terms[n]['run_directory']

input_dict = {}
input_dict['calculation_style'] = calculation_style
input_dict['lammps_command'] = run_terms[n]['lammps_command']
input_dict['mpi_command'] = run_terms[n]['mpi_command']
input_dict['potential_name'] = potential
input_dict['strainrange'] = '1e-8'

iprPy.highthroughput.prepare(dbase, run_directory, input_dict=input_dict)

### Run

In [36]:
iprPy.highthroughput.runner(dbase, run_directory)

### Analyze

Fetch records

In [78]:
results_df = dbase.get_records_df(style=record_style, full=True, flat=False)
results_df = results_df[results_df.potential_id == potential]
results_df = results_df[results_df.calc_script == 'calc_'+calculation_style]

print(len(results_df), 'records found')

60 records found


Check errors

In [79]:
error_df = results_df[results_df.status=='error']
print(len(error_df), 'issued errors')

if len(error_df) > 0:
    for error in np.unique(error_df.error.values):
        print(error)    

2 issued errors
Traceback (most recent call last):
  File "calc_LAMMPS_ELASTIC.py", line 450, in <module>
    main(*sys.argv[1:])
  File "calc_LAMMPS_ELASTIC.py", line 58, in main
    pressure_unit = input_dict['pressure_unit'])
  File "calc_LAMMPS_ELASTIC.py", line 170, in lammps_ELASTIC_refine
    dmax=dmax, pressure_unit=pressure_unit)
  File "calc_LAMMPS_ELASTIC.py", line 333, in lammps_ELASTIC
    output = lmp.run(lammps_command, 'in.elastic', mpi_command)
  File "c:\users\lmh1\documents\python-packages\atomman\atomman\lammps\run.py", line 98, in run
    raise ValueError('Invalid LAMMPS input: \n%s' % lines[-2])
ValueError: Invalid LAMMPS input: 
ERROR: Did not assign all restart atoms correctly (../read_restart.cpp:513)


Define composition

In [80]:
compositions = []
for i, calc in results_df.iterrows():
    compositions.append(comp_refine(calc.symbols, counts[calc.family]))
results_df = results_df.assign(composition=compositions)

#### Display $E_{coh}$ and lattice constants

In [81]:
# Unit conversions
results_df['a (A)'] = uc.get_in_units(results_df.final_a, 'Angstrom')
results_df['b (A)'] = uc.get_in_units(results_df.final_b, 'Angstrom')
results_df['c (A)'] = uc.get_in_units(results_df.final_c, 'Angstrom')
results_df['Ecoh (eV/atom)'] = uc.get_in_units(results_df.E_cohesive, 'eV')

# Loop over all compositions
for composition in np.unique(results_df.composition):
    show_df = results_df[results_df.composition==composition]
    
    print(composition)
    show_df = show_df[['family', 'Ecoh (eV/atom)', 'a (A)', 'b (A)', 'c (A)']].sort_values('Ecoh (eV/atom)')
    display(HTML(show_df.to_html(index=False)))

Al


family,Ecoh (eV/atom),a (A),b (A),c (A)
L1_0--AuCu,-3.360000,4.050000,4.050000,4.050000
D0_3--BiF3,-3.360000,8.100000,5.727565,5.727565
A6--In--bct,-3.360000,2.863782,2.863782,4.050000
L1_2--AuCu3,-3.360000,4.050000,4.050000,4.050000
A1--Cu--fcc,-3.360000,4.050000,4.050000,4.050000
B2--CsCl,-3.360000,2.863782,2.863782,4.050000
L2_1--AlCu2Mn--heusler,-3.360000,8.100000,5.727565,5.727565
A4--C--dc,-3.349106,2.852026,9.470456,4.939854
A15--Cr3Si,-3.346935,5.146685,5.146685,5.146685
A15--beta-W,-3.346935,5.146685,5.146685,5.146685


Al2Ni


family,Ecoh (eV/atom),a (A),b (A),c (A)
L2_1--AlCu2Mn--heusler,-4.510871,5.663980,5.663980,5.663980
C1--CaF2--fluorite,-3.914698,5.638257,5.638257,5.638257
L2_1--AlCu2Mn--heusler,-3.890008,7.599177,5.370156,5.370156
L2_1--AlCu2Mn--heusler,-3.836009,5.989147,5.989147,5.989147


Al3Ni


...

family,Ecoh (eV/atom),a (A),b (A),c (A)
L2_1--AlCu2Mn--heusler,-4.598788,5.542485,5.542485,5.542485
L2_1--AlCu2Mn--heusler,-4.598788,5.542485,5.542485,5.542485
L2_1--AlCu2Mn--heusler,-4.510871,5.663980,5.663980,5.663980
C1--CaF2--fluorite,-4.139251,5.421433,5.421433,5.421433


AlNi3


family,Ecoh (eV/atom),a (A),b (A),c (A)
L1_2--AuCu3,-4.631478,3.533170,3.533170,3.533170
D0_3--BiF3,-4.598788,5.542485,5.542485,5.542485
A15--Cr3Si,-4.553162,4.436320,4.436320,4.436320


Ni


family,Ecoh (eV/atom),a (A),b (A),c (A)
L2_1--AlCu2Mn--heusler,-4.450000,4.978032,4.978032,7.040000
A1--Cu--fcc,-4.450000,3.520000,3.520000,3.520000
A6--In--bct,-4.450000,2.489016,2.489016,3.520000
D0_3--BiF3,-4.450000,4.978032,4.978032,7.040000
L1_0--AuCu,-4.450000,3.520000,3.520000,3.520000
L1_2--AuCu3,-4.450000,3.520000,3.520000,3.520000
A3'--alpha-La--double-hcp,-4.438704,2.485424,4.304881,8.164521
A4--C--dc,-4.435272,2.480519,8.162225,4.314772
B3--ZnS--cubic-zinc-blende,-4.435272,2.480519,8.162225,4.314772
A3--Mg--hcp,-4.427868,2.481888,4.298757,4.104809


#### Display $C_{ij}$ for primary structure

In [83]:
for i, series in results_df[results_df.family == family].iterrows():
    C = series.C
    Cij = uc.get_in_units(C.Cij, 'GPa')
    print('Cij (GPa) for', series.composition, family, '=')
    for Ci in Cij:
        print('[%9.4f %9.4f %9.4f %9.4f %9.4f %9.4f]' % tuple(Ci))
    print()
    

Cij (GPa) for Ni A1--Cu--fcc =
[ 241.3213  150.8085  150.8085    0.0000    0.0000    0.0000]
[ 150.8085  241.3213  150.8085    0.0000    0.0000    0.0000]
[ 150.8085  150.8085  241.3213    0.0000    0.0000    0.0000]
[   0.0000    0.0000    0.0000  127.3403    0.0000    0.0000]
[   0.0000    0.0000    0.0000    0.0000  127.3403    0.0000]
[   0.0000    0.0000    0.0000    0.0000    0.0000  127.3403]

Cij (GPa) for Al A1--Cu--fcc =
[ 113.7833   61.5368   61.5368    0.0000    0.0000    0.0000]
[  61.5368  113.7833   61.5368    0.0000    0.0000    0.0000]
[  61.5368   61.5368  113.7833    0.0000    0.0000    0.0000]
[   0.0000    0.0000    0.0000   31.5984    0.0000    0.0000]
[   0.0000    0.0000    0.0000    0.0000   31.5984    0.0000]
[   0.0000    0.0000    0.0000    0.0000    0.0000   31.5984]



## 5. point_defect_static

In [84]:
n = 1
calculation_style = 'point_defect_static'
record_style = 'calculation_point_defect_formation'
sizemults = '10 10 10'
parent_calculation_style = 'LAMMPS_ELASTIC'
parent_record_style = 'calculation_system_relax'

### Prepare

Get parent records

In [85]:
parent_df = dbase.get_records_df(style=parent_record_style, full=True, flat=False)
parent_df = parent_df[parent_df.potential_id == potential]
parent_df = parent_df[parent_df.calc_script == 'calc_'+parent_calculation_style]
parent_df = parent_df[parent_df.family == family]
parent_records = dbase.get_records(name=parent_df.calc_key.tolist())

print(len(parent_records), 'parent records found')

2 parent records found


Define prepare function variables

In [86]:
run_directory = run_terms[n]['run_directory']

input_dict = {}
input_dict['calculation_style'] = calculation_style
input_dict['lammps_command'] = run_terms[n]['lammps_command']
input_dict['mpi_command'] = run_terms[n]['mpi_command']
input_dict['sizemults'] = sizemults
input_dict['parent_records'] = parent_records

Call prepare

In [87]:
iprPy.highthroughput.prepare(dbase, run_directory, input_dict=input_dict)

### Run

In [88]:
iprPy.highthroughput.runner(dbase, run_directory)

### Analyze

Fetch records

In [89]:
results_df = dbase.get_records_df(style=record_style, full=True, flat=False)
results_df = results_df[results_df.potential_id == potential]
results_df = results_df[results_df.calc_script == 'calc_'+calculation_style]
results_df['parent_key'] = [os.path.splitext(parent_key)[0] for parent_key in results_df.load_file]
results_df = results_df[results_df.parent_key.isin(parent_df.calc_key)]

print(len(results_df), 'records found')

18 records found


Check errors

In [90]:
error_df = results_df[results_df.status=='error']
print(len(error_df), 'issued errors')

if len(error_df) > 0:
    for error in np.unique(error_df.error.values):
        print(error)    

0 issued errors


Define composition

In [91]:
compositions = []
for i, calc in results_df.iterrows():
    compositions.append(comp_refine(calc.symbols, counts[calc.family]))
results_df = results_df.assign(composition=compositions)

#### Display $E^{ptd}_f$

In [92]:
# Unit conversions
results_df['E_f (eV)'] = uc.get_in_units(results_df.E_f, 'eV')

# Loop over all compositions
for composition in np.unique(results_df.composition):
    show_df = results_df[results_df.composition==composition]
    
    print(composition)
    show_df = show_df[['pointdefect_id', 'E_f (eV)', 'reconfigured']].sort_values('E_f (eV)')
    display(HTML(show_df.to_html(index=False)))

Al


pointdefect_id,E_f (eV),reconfigured
A1--Cu--fcc--vacancy,0.675330,False
A1--Cu--fcc--1nn-divacancy,1.361453,False
A1--Cu--fcc--2nn-divacancy,1.377990,False
A1--Cu--fcc--100-dumbbell,2.586203,False
A1--Cu--fcc--octahedral-interstitial,2.786985,False
A1--Cu--fcc--crowdion-interstitial,2.899777,False
A1--Cu--fcc--110-dumbbell,2.906438,False
A1--Cu--fcc--111-dumbbell,3.001311,False
A1--Cu--fcc--tetrahedral-interstitial,3.089378,False


Ni


pointdefect_id,E_f (eV),reconfigured
A1--Cu--fcc--vacancy,1.571367,False
A1--Cu--fcc--1nn-divacancy,2.975356,False
A1--Cu--fcc--2nn-divacancy,3.125092,False
A1--Cu--fcc--100-dumbbell,3.950914,False
A1--Cu--fcc--111-dumbbell,4.218501,False
A1--Cu--fcc--110-dumbbell,4.283308,False
A1--Cu--fcc--crowdion-interstitial,4.283838,False
A1--Cu--fcc--octahedral-interstitial,4.309433,False
A1--Cu--fcc--tetrahedral-interstitial,4.430902,False


## 6. surface_energy

In [42]:
n = 1
calculation_style = 'surface_energy'
record_style = 'calculation_surface_energy'
sizemults = '5 5 10'
parent_calculation_style = 'LAMMPS_ELASTIC'
parent_record_style = 'calculation_system_relax'

### Prepare

Get parent records

In [60]:
parent_df = dbase.get_records_df(style=parent_record_style, full=True, flat=False)
parent_df = parent_df[parent_df.potential_id == potential]
parent_df = parent_df[parent_df.calc_script == 'calc_'+parent_calculation_style]
parent_df = parent_df[parent_df.family == family]
parent_records = dbase.get_records(name=parent_df.calc_key.tolist())

print(len(parent_records), 'parent records found')

2 parent records found


Define prepare function variables

In [61]:
run_directory = run_terms[n]['run_directory']

input_dict = {}
input_dict['calculation_style'] = calculation_style
input_dict['lammps_command'] = run_terms[n]['lammps_command']
input_dict['mpi_command'] = run_terms[n]['mpi_command']
input_dict['sizemults'] = sizemults
input_dict['parent_records'] = parent_records

Call prepare

In [62]:
iprPy.highthroughput.prepare(dbase, run_directory, input_dict=input_dict)

### Run

In [63]:
iprPy.highthroughput.runner(dbase, run_directory)

### Analyze

Fetch records

In [64]:
results_df = dbase.get_records_df(style=record_style, full=True, flat=False)
results_df = results_df[results_df.potential_id == potential]
results_df = results_df[results_df.calc_script == 'calc_'+calculation_style]
results_df['parent_key'] = [os.path.splitext(parent_key)[0] for parent_key in results_df.load_file]
results_df = results_df[results_df.parent_key.isin(parent_df.calc_key)]

print(len(results_df), 'records found')

6 records found


Check errors

In [65]:
error_df = results_df[results_df.status=='error']
print(len(error_df), 'issued errors')

if len(error_df) > 0:
    for error in np.unique(error_df.error.values):
        print(error)    

0 issued errors


Define composition

In [66]:
compositions = []
for i, calc in results_df.iterrows():
    compositions.append(comp_refine(calc.symbols, counts[calc.family]))
results_df = results_df.assign(composition=compositions)

#### Display $\gamma_{fs}$

In [76]:
# Unit conversions
results_df['gamma_fs (mJ/m^2)'] = uc.get_in_units(results_df.gamma_fs, 'mJ/m^2')

# Loop over all compositions
for composition in np.unique(results_df.composition):
    show_df = results_df[results_df.composition==composition]
    
    print(composition)
    show_df = show_df[['surface_id', 'gamma_fs (mJ/m^2)']].sort_values('gamma_fs (mJ/m^2)')
    display(HTML(show_df.to_html(index=False)))

Al


surface_id,gamma_fs (mJ/m^2)
A1--Cu--fcc--111,870.521179
A1--Cu--fcc--100,943.590331
A1--Cu--fcc--110,1006.026801


Ni


surface_id,gamma_fs (mJ/m^2)
A1--Cu--fcc--111,1759.096099
A1--Cu--fcc--100,1936.035517
A1--Cu--fcc--110,2087.076695


## 7. Stacking Fault

In [93]:
n = 1
calculation_style = 'stacking_fault_multi'
record_style = 'calculation_generalized_stacking_fault'
sizemults = '5 5 10'
numshifts1 = 51 
numshifts2 = 51
parent_calculation_style = 'LAMMPS_ELASTIC'
parent_record_style = 'calculation_system_relax'

### Prepare

Get parent records

In [94]:
parent_df = dbase.get_records_df(style=parent_record_style, full=True, flat=False)
parent_df = parent_df[parent_df.potential_id == potential]
parent_df = parent_df[parent_df.calc_script == 'calc_'+parent_calculation_style]
parent_df = parent_df[parent_df.family == family]
parent_records = dbase.get_records(name=parent_df.calc_key.tolist())

print(len(parent_records), 'parent records found')

2 parent records found


Define prepare function variables

In [95]:
run_directory = run_terms[n]['run_directory']

input_dict = {}
input_dict['calculation_style'] = calculation_style
input_dict['lammps_command'] = run_terms[n]['lammps_command']
input_dict['mpi_command'] = run_terms[n]['mpi_command']
input_dict['sizemults'] = sizemults
input_dict['stackingfault_numshifts1'] = str(numshifts1)
input_dict['stackingfault_numshifts2'] = str(numshifts2)
input_dict['parent_records'] = parent_records

Call prepare

In [96]:
iprPy.highthroughput.prepare(dbase, run_directory, input_dict=input_dict)

### Run

In [97]:
iprPy.highthroughput.runner(dbase, run_directory)

### Analyze

Fetch records

In [98]:
results_df = dbase.get_records_df(style=record_style, full=True, flat=False)
results_df = results_df[results_df.potential_id == potential]
results_df = results_df[results_df.calc_script == 'calc_'+calculation_style]
results_df['parent_key'] = [os.path.splitext(parent_key)[0] for parent_key in results_df.load_file]
results_df = results_df[results_df.parent_key.isin(parent_df.calc_key)]

print(len(results_df), 'records found')

2 records found


Check errors

In [99]:
error_df = results_df[results_df.status=='error']
print(len(error_df), 'issued errors')

if len(error_df) > 0:
    for error in np.unique(error_df.error.values):
        print(error)    

0 issued errors


Define composition

In [100]:
compositions = []
for i, calc in results_df.iterrows():
    compositions.append(comp_refine(calc.symbols, counts[calc.family]))
results_df = results_df.assign(composition=compositions)

Plot $\gamma_{gsf}$

In [101]:
# Loop over records
for i, series in results_df.iterrows():
    print(series.composition)
    
    plot_df = series.gsf_plot
    plot_df['gamma_gsf (mJ/m^2)'] = uc.get_in_units(plot_df.energy, 'mJ/m^2')
    
    shift1_df = plot_df[np.isclose(plot_df.shift2, 0.0)]
    p = figure(title='Generalized Stacking Fault',
              plot_width = 800,
              plot_height = 600,
              x_axis_label = 'Reduced Coordinate',
              y_axis_label = 'Stacking Fault Energy (mJ/m^2)')
    p.line(shift1_df.shift1, shift1_df['gamma_gsf (mJ/m^2)'],
          line_color='black', line_width=2)
    show(p)
    
    shift2_df = plot_df[np.isclose(plot_df.shift1, 0.0)]    
    p = figure(title='Generalized Stacking Fault',
              plot_width = 800,
              plot_height = 600,
              x_axis_label = 'Reduced Coordinate',
              y_axis_label = 'Stacking Fault Energy (mJ/m^2)')
    p.line(shift2_df.shift2, shift2_df['gamma_gsf (mJ/m^2)'],
          line_color='black', line_width=2)
    show(p)    

Al


Ni


## 8. dislocation_monopole

In [102]:
n = 4
calculation_style = 'dislocation_monopole'
record_style =      'calculation_dislocation_monopole'
annealtemperature = 50
parent_calculation_style = 'LAMMPS_ELASTIC'
parent_record_style = 'calculation_system_relax'

sizemults = {}
sizemults['A1--Cu--fcc--110-screw'] = '-28 28 -40 40 0 2'
sizemults['A1--Cu--fcc--111-edge'] = '-49 49 -40 40 0 2'
sizemults['A2--W--bcc--110-edge'] = '-40 40 -28 28 0 2'
sizemults['A2--W--bcc--111-screw'] = '-28 28 -49 49 0 2'
sizemults['A2--W--bcc--112-edge'] = '-40 40 -49 49 0 2'

### Prepare

Get parent records

In [103]:
parent_df = dbase.get_records_df(style=parent_record_style, full=True, flat=False)
parent_df = parent_df[parent_df.potential_id == potential]
parent_df = parent_df[parent_df.calc_script == 'calc_'+parent_calculation_style]
parent_df = parent_df[parent_df.family == family]
parent_records = dbase.get_records(name=parent_df.calc_key.tolist())

print(len(parent_records), 'parent records found')

2 parent records found


Define prepare function variables and call prepare

In [104]:
run_directory = run_terms[n]['run_directory']

for dislocation_name in sizemults.keys():
    input_dict = {}
    input_dict['calculation_style'] = 'dislocation_monopole'
    input_dict['lammps_command'] = run_terms[n]['lammps_command']
    input_dict['mpi_command'] = run_terms[n]['mpi_command']
    input_dict['annealtemperature'] = str(annealtemperature)
    input_dict['parent_records'] = parent_records
    input_dict['dislocation_name'] = dislocation_name
    input_dict['sizemults'] = sizemults[dislocation_name]
    
    iprPy.highthroughput.prepare(dbase, run_directory, input_dict=input_dict)

### Run

In [43]:
iprPy.highthroughput.runner(dbase, run_directory)

### Analyze

Fetch records

In [98]:
results_df = dbase.get_records_df(style=record_style, full=True, flat=False)
results_df = results_df[results_df.potential_id == potential]
results_df = results_df[results_df.calc_script == 'calc_'+calculation_style]
results_df['parent_key'] = [os.path.splitext(parent_key)[0] for parent_key in results_df.load_file]
results_df = results_df[results_df.parent_key.isin(parent_df.calc_key)]

print(len(results_df), 'records found')

2 records found


Check errors

In [99]:
error_df = results_df[results_df.status=='error']
print(len(error_df), 'issued errors')

if len(error_df) > 0:
    for error in np.unique(error_df.error.values):
        print(error)    

0 issued errors
